In [1]:
# Proof of concept implementation

In [2]:
from pytorch_pretrained_vit import ViT
import json
from PIL import Image
import torch
import torchvision
from torchvision import transforms
import cv2

In [3]:
# Load data
#img = Image.open('img.jpg')

#transform = transforms.Compose(
#    [transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#batch_size = 1

# testset = torchvision.datasets.ImageNet(root='./data', train=False, download=True, transform=transform)
#testset = torchvision.datasets.FakeData(100,(3,224,224), 10, transform=transform)

#testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [4]:
img = Image.open('img.jpg')

In [5]:
#import matplotlib.pyplot as plt
#import numpy as np


# Show image
#def imshow(img):
#    img = img / 2 + 0.5     # unnormalize
#    npimg = img.numpy()
#    plt.imshow(np.transpose(npimg, (1, 2, 0)))
#    plt.show()


# get some random training images
#dataiter = iter(testloader)
#image, label = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(image))
# print labels
#print(' '.join('%5s' % classes[label[j]] for j in range(batch_size)))

In [6]:
# Load pretrained Vision Transformer
model = ViT('B_16_imagenet1k', pretrained=True)

Loaded pretrained weights.


In [7]:
# Object detection
detector_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).autoshape()
results = detector_model(img, size=640)
results.print()
results.show()

Using cache found in /Users/lautenschlager/.cache/torch/hub/ultralytics_yolov5_master
Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2021-5-23 torch 1.8.1 CPU



AutoShape already enabled, skipping... 
image 1/1: 563x845 1 bird
Speed: 10.6ms pre-process, 186.8ms inference, 1.3ms NMS per image at shape (1, 3, 448, 640)


In [8]:
# Get bounding box coordinates from detection
x1 = int(results.xyxy[0][0][0])
y1 = int(results.xyxy[0][0][1])
x2 = int(results.xyxy[0][0][2])
y2 = int(results.xyxy[0][0][3])
bbox_points=[x1, y1, x2, y2]
print('bounding box is:', x1, y1, x2, y2)

bounding box is: 253 87 587 541


In [9]:
# Show cropped image (bounding box)
cropped_img = img.crop((x1, y1, x2, y2))
cropped_img.show()
type(cropped_img)

PIL.Image.Image

In [10]:
# Preprocess image for transformer
tfms = transforms.Compose([transforms.Resize(model.image_size), transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),])
preprocessed_image = tfms(cropped_img).unsqueeze(0)

In [16]:
# Segmentation
# Code adapted from https://github.com/spmallick/learnopencv/blob/master/app-seperation-semseg/Background-Removal.py
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2


# %matplotlib.use('TkAgg')

# Apply the transformations needed
import torchvision.transforms as T

# Define the helper function
def decode_segmap(image, source, nc=21):
  
#  label_colors = np.array([(0, 0, 0),  # 0=background
#               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
#               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
#               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
#               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
#               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
#               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
#               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
#               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])
#
#  r = np.zeros_like(image).astype(np.uint8)
#  g = np.zeros_like(image).astype(np.uint8)
#  b = np.zeros_like(image).astype(np.uint8)
#   
#
#  for l in range(0, nc):
#    idx = image == l
#    r[idx] = label_colors[l, 0]
#    g[idx] = label_colors[l, 1]
#    b[idx] = label_colors[l, 2]
# 
#  rgb = np.stack([r, g, b], axis=2)

  # Load the foreground input image 
  foreground = source

  # Change the color of foreground image to RGB 
  # and resize image to match shape of R-band in RGB output map
  #foreground = cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB)
  
  #foreground = cv2.resize(np.array(foreground),(r.shape[1],r.shape[0]))

  # Create a background array to hold white pixels
  # with the same size as RGB output map
  background = 255 * np.ones_like(rgb).astype(np.uint8)

  # Convert uint8 to float
  foreground = foreground.astype(float)
  background = background.astype(float)

  # Create a binary mask of the RGB output map using the threshold value 0
  th, alpha = cv2.threshold(np.array(rgb),0,255, cv2.THRESH_BINARY)

  # Apply a slight blur to the mask to soften edges
  alpha = cv2.GaussianBlur(alpha, (7,7),0)

  # Normalize the alpha mask to keep intensity between 0 and 1
  alpha = alpha.astype(float)/255

  # Multiply the foreground with the alpha matte
  foreground = cv2.multiply(alpha, foreground)  
  
  # Multiply the background with ( 1 - alpha )
  background = cv2.multiply(1.0 - alpha, background)  
  
  # Add the masked foreground and background
  outImage = cv2.add(foreground, background)

  # Return a normalized output image for display
  return outImage/255

def segment(net, image, show_orig=True, dev='cpu'):
  img = image
  if show_orig: plt.imshow(img); plt.axis('off'); plt.show()
  # Comment the Resize and CenterCrop for better inference results
  trf = T.Compose([T.Resize(450), 
                   #T.CenterCrop(224), 
                   T.ToTensor(), 
                   T.Normalize(mean = [0.485, 0.456, 0.406], 
                               std = [0.229, 0.224, 0.225])])
  inp = trf(img).unsqueeze(0).to(dev)
  out = net.to(dev)(inp)['out']
  om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
  
  rgb = decode_segmap(om, img)
    
  plt.imshow(rgb); plt.axis('off'); plt.show()

  plt.savefig(fname='seg_result')
  
dlab = models.segmentation.deeplabv3_resnet101(pretrained=1).eval()

segment(dlab, cropped_img, show_orig=False)


NameError: name 'rgb' is not defined

In [12]:
# Load class labels
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

In [13]:
# Classify
model.eval()
with torch.no_grad():
    outputs = model(preprocessed_image).squeeze(0)
print('-----')  
for idx in torch.topk(outputs, k=3).indices.tolist():
    prob = torch.softmax(outputs, -1)[idx].item()
    print('[{idx}] {label:<75} ({p:.2f}%)'.format(idx=idx, label=labels_map[idx], p=prob*100))

-----
[24] great grey owl, great gray owl, Strix nebulosa                              (99.98%)
[21] kite                                                                        (0.00%)
[798] slide rule, slipstick                                                       (0.00%)
